Решить методами Гаусса и Зейделя, найти $\lambda_{min}$ и $\lambda_{max}$, определить число обусловленности матрицы $\mu = \|A\| \cdot \|A^{-1}\|$. Сделать печать невязок обоих методов. Указать критерий останова итераций метода Зейделя.

In [151]:
from math import sqrt, inf
import numpy as np
from numpy.linalg import inv
from scipy.spatial.distance import euclidean

def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

a = [1] * 19
b = [-2] * 20
c = [1] * 19

A = tridiag(a, b, c)
A[0] = [1] + [0]*19
A[-1] = [1] + [2]*18 + [1]

b = np.array([1] + [2/i**2 for i in range(2, 21)])

print(A)
print(A.shape)

[[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1 -2  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 -2  1  0

## 1. Метод Гаусса

In [0]:
def gaussian_elimination(A, b):
    n = len(A)
    
    if len(b) != n:
        raise ValueError("Различаются размеры A и b", n, len(b))
        
    for first_row in range(n - 1):
        for row in range(first_row + 1, n):
            multiplier = A[row][first_row] / A[first_row][first_row]
            A[row][first_row] = multiplier
            for col in range(first_row + 1, n):
                A[row][col] = A[row][col] - multiplier*A[first_row][col]
            b[row] = b[row] - multiplier*b[first_row]

    x = np.zeros(n)
    k = n - 1
    x[k] = b[k] / A[k, k]
    
    while k >= 0:
        x[k] = (b[k] - np.dot(A[k, k+1:], x[k+1:])) / A[k, k]
        k = k - 1

    return x

In [80]:
x = gaussian_elimination(A.copy(), b.copy())
error = np.dot(A, x) - b
print(x)

print(f"Невязка: {error}")

[ 1.          0.0857316  -0.32853681 -0.35631459 -0.25909236 -0.08187014
  0.15090764  0.42450174  0.72934585]
Невязка: [ 0.00000000e+00  0.00000000e+00  1.64268404e-01  0.00000000e+00
  6.93889390e-17 -2.77555756e-17  4.85722573e-17  0.00000000e+00
  4.44089210e-16]


## 2. Метод Зейделя

$$\mathbf {x} ^{(k+1)}=-(L+D)^{-1} U\mathbf {x} ^{(k)} + (L+D)^{-1}\mathbf {b} =  K\mathbf {x} ^{(k)} + F$$

$$\text{В качестве критерия останова будем использовать }\parallel Ax^{(k)}-b\parallel \le \varepsilon$$

In [0]:
L = np.tril(A, k=-1)
D = np.diag(np.diag(A))
U = np.triu(A, k=1)

In [193]:
Q = np.linalg.inv(D + L)
K = -np.dot(Q,U)
F = np.dot(Q, b)

x = np.zeros_like(b)
for j in range(1, ITERATION_LIMIT):
    x_new = np.dot(K, x) + F
    if np.linalg.norm(np.dot(A, x) - b) < 1e-15:
#     if np.allclose(x, x_new, rtol=1e-20):
        print('Потребовалось ', j, ' итераций')
        break
    x = x_new

error = np.dot(A, x) - b
print(f"Невязка: {error}")

Потребовалось  371  итераций
Невязка: [ 0.00000000e+00  0.00000000e+00 -2.22044605e-16  0.00000000e+00
 -3.74700271e-16 -4.16333634e-16 -1.73472348e-16 -2.22044605e-16
 -1.11022302e-16 -9.36750677e-17 -5.55111512e-17  6.24500451e-17
  1.26634814e-16  1.78676518e-16  3.90312782e-16  2.22044605e-16
  1.05818132e-16  4.71844785e-16 -5.29090660e-17  1.15359111e-16]


#### Собственные значения и число обусловленности

In [187]:
print(max(np.linalg.eigvals(K)).real, min(np.linalg.eigvals(K)).real)

0.9119637897545003 -0.4060820436489216


$\lambda_{max} \approx 0.9119637897545003,\ \lambda_{min} \approx -0.4060820436489216$

In [191]:
print('Число обусловленности: ', np.linalg.cond(A))

Число обусловленности:  238.725557642878


Получили, что наша задача не очень хорошо обусловлена. В частности из-за этого решения полученные 2 алгоритмами сильно отличаются.

In [194]:
print('Метод Гаусса: ', gaussian_elimination(A.copy(), b.copy()))
print('Метод Зейделя: ', x)

Метод Гаусса:  [ 1.         -0.78272809 -2.06545618 -2.09323396 -1.99601174 -1.81878952
 -1.58601174 -1.31241763 -1.00757353 -0.67803807 -0.3285026   0.03756178
  0.41751506  0.80930266  1.21129433  1.6221749   2.04086797  2.46648145
  2.89826777  3.33559426]
Метод Зейделя:  [ 1.          0.04402653 -0.41194694 -0.64569819 -0.75444943 -0.78320068
 -0.75639637 -0.68877574 -0.5899051  -0.46634311 -0.32278112 -0.1626902
  0.01128961  0.19710373  0.39312194  0.59802904  0.81074864  1.03038865
  1.2562015   1.48755452]


## 3. Метод последовательной верхней релаксации

$$\mathbf {x} ^{(k+1)}=(D+\omega L)^{-1}{\big ((1 - \omega)D - \omega U {\big )}\mathbf {x} ^{(k)} + \omega (D+\omega L)^{-1}} \mathbf {b} = K\mathbf {x} ^{(k)} + F$$

In [178]:
w = 1.07
Q = np.linalg.inv(D + w * L)
K = np.dot(Q,((1.0 - w) * D - w * U))
F = w*np.dot(Q, b)

x = np.zeros_like(b)
x_1 = np.dot(K, x) + F
n = len(b)
for j in range(1, ITERATION_LIMIT):
    x_new = np.dot(K, x) + F
    if np.linalg.norm(np.dot(A, x) - b) < 1e-15:
        print('Потребовалось ', j, ' итераций')
        break
    x = x_new

error = np.dot(A, x) - b
print(f"Невязка: {error}")

Потребовалось  332  итераций
Невязка: [-1.11022302e-16  1.11022302e-16 -2.22044605e-16 -2.22044605e-16
 -1.52655666e-16  2.77555756e-17 -1.73472348e-16  2.22044605e-16
  3.33066907e-16  2.39391840e-16  2.77555756e-16  3.40005801e-16
  2.27248775e-16  2.34187669e-16 -5.37764278e-17  0.00000000e+00
 -3.38271078e-16 -4.16333634e-16 -5.29090660e-17 -1.06685494e-16]


Чтобы подобрать оптимальное значение $w$, воспользуемся байесовской оптимизацией

In [167]:
from hyperopt import fmin, tpe, rand, hp

# Будем искать оптимальное w на промежутке (1.0, 1.2)
# после w=1.2 на данной матрице алгоритм перестаёт сходиться
space = {'w': hp.uniform('w', 1.0, 1.2)}

def f(params):
    w = params['w']
    Q = np.linalg.inv(D + w * L)
    K = np.dot(Q,((1.0 - w) * D - w * U))
    F = w*np.dot(Q, b)
    x = np.zeros_like(b)
    x_1 = np.dot(K, x) + F
    n = len(b)
    for j in range(1, ITERATION_LIMIT):
        x_new = np.dot(K, x) + F
        if np.linalg.norm(np.dot(A, x) - b) < 1e-15:
            return j
        x = x_new
    return inf

best = fmin(fn=f, space=space, algo=tpe.suggest, max_evals=1000)
print(best)

{'w': 1.1477197729145185}


Данный метод позволил снизить число итераций с 371 до 305 (примерно на 18%)

In [179]:
w = best['w']
Q = np.linalg.inv(D + w * L)
K = np.dot(Q,((1.0 - w) * D - w * U))
F = w*np.dot(Q, b)

x = np.zeros_like(b)
x_1 = np.dot(K, x) + F
n = len(b)
for j in range(1, ITERATION_LIMIT):
    x_new = np.dot(K, x) + F
    if np.linalg.norm(np.dot(A, x) - b) < 1e-15:
        print('Потребовалось ', j, ' итераций')
        break
    x = x_new

error = np.dot(A, x) - b
print(f"Невязка: {error}")

Потребовалось  305  итераций
Невязка: [ 4.44089210e-16  2.22044605e-16  0.00000000e+00  0.00000000e+00
  6.93889390e-17  2.77555756e-17  4.85722573e-17 -2.22044605e-16
 -1.11022302e-16 -9.36750677e-17 -1.66533454e-16 -4.85722573e-17
 -2.79290480e-16 -4.33680869e-17 -5.37764278e-17  0.00000000e+00
  1.05818132e-16  4.71844785e-16 -5.29090660e-17  1.15359111e-16]
